In [1]:
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Flatten
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard

In [2]:
file = open("nietzsche.txt", encoding="utf-8")
text_niet = file.read()
# 732020:732176

file_date = open("baddate.txt", encoding="utf-8")
text_date = file_date.read()

EPOCHS = 50
diversity = 0.5

# Do we not
# hear the noise of the grave-diggers who are burying God? Do we not smell
# the divine putrefaction?—for even Gods putrefy! God is dead! God remains
# dead! And we have killed him!


In [3]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)
    preds = preds / np.sum(preds)               
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)
    


In [4]:
chars = sorted(list(set(text_niet)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(40, len(text_niet) - seqlen - 1, step):
    sentences.append(text_niet[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1
        
model1 = Sequential()
model1.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model1.add(Dense(len(chars), activation='softmax'))

model1.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def on_epoch_end_niet(epoch, _):
    """Function invoked at end of each epoch."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text_niet) - seqlen - 1)

    generated = ''
    sentence = text_niet[start_index: start_index + seqlen]
    generated += sentence
    #print('----- Generating with seed: "' + sentence + '"')
    #sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        
        preds = model1.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        #sys.stdout.write(next_char) 

    # for diversity in [0.2, 0.5, 1.0]:
    #     #print('----- diversity:', diversity)

    #     generated = ''
    #     sentence = text_niet[start_index: start_index + seqlen]
    #     generated += sentence
    #     #print('----- Generating with seed: "' + sentence + '"')
    #     #sys.stdout.write(generated)

    #     for i in range(400):
    #         x_pred = np.zeros((1, seqlen, len(chars)))
    #         for t, char in enumerate(sentence):
    #             x_pred[0, t, char_indices[char]] = 1.
            
    #         preds = model1.predict(x_pred, verbose=0)
    #         next_index = sample(preds[0, -1], diversity)
    #         next_char = indices_char[next_index]

    #         sentence = sentence[1:] + next_char

    #         #sys.stdout.write(next_char)   

print_callback = LambdaCallback(on_epoch_end=on_epoch_end_niet)


model1.fit(x, y,
          batch_size=128,
          epochs=EPOCHS,
          callbacks=[print_callback])


C:\Users\timjh\AppData\Local\Temp\ipykernel_8520\1356494578.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
C:\Users\timjh\AppData\Local\Temp\ipykernel_8520\1356494578.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), seqlen, len(chars)), dtype=n

Epoch 1/50
666/667 [============================>.] - ETA: 0s - loss: 2.0099 - categorical_crossentropy: 2.0099 - accuracy: 0.4286
----- Generating text after Epoch: 0
667/667 [==============================] - 93s 136ms/step - loss: 2.0097 - categorical_crossentropy: 2.0097 - accuracy: 0.4286
Epoch 2/50
666/667 [============================>.] - ETA: 0s - loss: 1.5755 - categorical_crossentropy: 1.5755 - accuracy: 0.5378
----- Generating text after Epoch: 1
667/667 [==============================] - 88s 132ms/step - loss: 1.5755 - categorical_crossentropy: 1.5755 - accuracy: 0.5378
Epoch 3/50
666/667 [============================>.] - ETA: 0s - loss: 1.5127 - categorical_crossentropy: 1.5127 - accuracy: 0.5534
----- Generating text after Epoch: 2
667/667 [==============================] - 93s 139ms/step - loss: 1.5126 - categorical_crossentropy: 1.5126 - accuracy: 0.5534
Epoch 4/50
666/667 [============================>.] - ETA: 0s - loss: 1.4845 - categorical_crossentropy: 1.4845 - a

C:\Users\timjh\AppData\Local\Temp\ipykernel_8520\748897970.py:4: RuntimeWarning: divide by zero encountered in log
  preds = np.exp(np.log(preds) / temperature)


667/667 [==============================] - 315s 472ms/step - loss: 1.3988 - categorical_crossentropy: 1.3988 - accuracy: 0.5800
Epoch 26/50
667/667 [==============================] - ETA: 0s - loss: 1.3978 - categorical_crossentropy: 1.3978 - accuracy: 0.5805
----- Generating text after Epoch: 25
667/667 [==============================] - 320s 479ms/step - loss: 1.3978 - categorical_crossentropy: 1.3978 - accuracy: 0.5805
Epoch 27/50
667/667 [==============================] - ETA: 0s - loss: 1.3971 - categorical_crossentropy: 1.3971 - accuracy: 0.5807
----- Generating text after Epoch: 26
667/667 [==============================] - 319s 478ms/step - loss: 1.3971 - categorical_crossentropy: 1.3971 - accuracy: 0.5807
Epoch 28/50
667/667 [==============================] - ETA: 0s - loss: 1.3962 - categorical_crossentropy: 1.3962 - accuracy: 0.5810
----- Generating text after Epoch: 27
667/667 [==============================] - 329s 494ms/step - loss: 1.3962 - categorical_crossentropy: 1.39

In [8]:
chars_date = sorted(list(set(text_date)))
char_indices_date = dict((c, i) for i, c in enumerate(chars_date))
indices_char_date = dict((i, c) for i, c in enumerate(chars_date))

seqlen = 40
step = seqlen
sentences = []
for i in range(40, len(text_date) - seqlen - 1, step):
    sentences.append(text_date[i: i + seqlen + 1])

x_d = np.zeros((len(sentences), seqlen, len(chars_date)), dtype=np.bool)
y_d = np.zeros((len(sentences), seqlen, len(chars_date)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x_d[i, t, char_indices_date[char_in]] = 1
        y_d[i, t, char_indices_date[char_out]] = 1

model2 = Sequential()
model2.add(LSTM(128, input_shape=(seqlen, len(chars_date)), return_sequences=True))
model2.add(Dense(len(chars_date), activation='softmax'))

model2.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def on_epoch_end_date(epoch, _):
    """Function invoked at end of each epoch."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text_date) - seqlen - 1)

    generated = ''
    sentence = text_date[start_index: start_index + seqlen]
    generated += sentence
    #print('----- Generating with seed: "' + sentence + '"')
    #sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars_date)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices_date[char]] = 1.
        
        preds = model2.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char_date[next_index]

        sentence = sentence[1:] + next_char

print_callback = LambdaCallback(on_epoch_end=on_epoch_end_date)

model2.fit(x_d, y_d,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

C:\Users\timjh\AppData\Local\Temp\ipykernel_8520\798493350.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_d = np.zeros((len(sentences), seqlen, len(chars_date)), dtype=np.bool)
C:\Users\timjh\AppData\Local\Temp\ipykernel_8520\798493350.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_d = np.zeros((len(sentences), seqlen, len(chars_dat

Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 3.5863 - categorical_crossentropy: 3.5863 - accuracy: 0.1443
----- Generating text after Epoch: 0
21/21 [==============================] - 38s 2s/step - loss: 3.5863 - categorical_crossentropy: 3.5863 - accuracy: 0.1443
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 3.0744 - categorical_crossentropy: 3.0744 - accuracy: 0.2218
----- Generating text after Epoch: 1
21/21 [==============================] - 35s 2s/step - loss: 3.0744 - categorical_crossentropy: 3.0744 - accuracy: 0.2218
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 2.5464 - categorical_crossentropy: 2.5464 - accuracy: 0.3161
----- Generating text after Epoch: 2
21/21 [==============================] - 36s 2s/step - loss: 2.5464 - categorical_crossentropy: 2.5464 - accuracy: 0.3161
Epoch 4/20
21/21 [==============================] - ETA: 0s - loss: 2.2989 - categorical_crossentropy: 2.2989 - accuracy: 0.3622
----- G

In [ ]:
chars_date = sorted(list(set(text_date)))
char_indices_date = dict((c, i) for i, c in enumerate(chars_date))
indices_char_date = dict((i, c) for i, c in enumerate(chars_date))


In [13]:
seqlenn = 40

def generate_date(seed):
    output = ""
    print()
    rand_evn = random.randint(0,100)

    sentence = seed
    generated = ''
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlenn, len(chars_date)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices_date[char]] = 1.
        
        preds = model2.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], 0.7)
        next_char = indices_char_date[next_index]

        sentence = sentence[1:] + next_char
        output = output + next_char
        sys.stdout.write(next_char)
    return output

def generate_niet(seed):
    output = ""
    print()
    sentence = seed
    generated = ''

    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(200):
        x_pred = np.zeros((1, seqlenn, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        
        preds = model1.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], 0.7)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char
        output = output + next_char
        sys.stdout.write(next_char)
    return output

output = text_niet[732020:732176]

for i in range(0,10):
    if i % 2 == 0:
        print("\nGenerating Hallmark Text\n")
        output = generate_date(output[:40])
    else:
        print("\nGenerating Nietzsche Text\n")
        output = generate_niet(output[:40])







----- Generating with seed: "Do we not
hear the noise of the grave-di"
Do we not
hear the noise of the grave-ding to the back to proble this not.
LEIGH
So, I me. This is a goom from *
BRAD *
Hell, tome her and
staying each to the company.
ALLISON *
(reads)
“Pet wer. *
Leigh and Conner and then off the end to we staof the crommmpacty.
96 16 EXT. LEIGH AND ERIN’S APATT CONTINEES - DAY
Establishing shot of the retel can.
119111 11D ALLISON’S OFFICE - DAY
Leigh and Erin ar un on the coffee for
them. 